# Mapping Agricultural Land Suitability using Soil Properties in Sonipat District. 

In [1]:
# create folder to save output
import os
# set the basepath - folder containing data and where the ouput will be written
basepath = os.path.join(os.getcwd(), "Output")
os.makedirs(basepath, exist_ok=True)

## Explore the data

In [ ]:
#path to input data (soil properties layer in Sonipat)
data = "path to geopackage"

In [1]:
import os
import geopandas as gpd
soil_prop_gdf = gpd.read_file(data)
#print first 5 rows
soil_prop_gdf.head(5)

,Depth,Surface_te,pH,Drainage,Slope,geometry
0,Deep,Loamy,Slightly alkaline,Well,Level to nearly level,"MULTIPOLYGON (((3734219.156 4537926.928, 37343..."
1,Deep,Loamy,Neutral,Well,Level to nearly level,"MULTIPOLYGON (((3733766.495 4533437.457, 37336..."
2,Deep,Loamy,Slightly alkaline,Moderately well,Very gently sloping,"MULTIPOLYGON (((3731220.970 4531162.026, 37312..."
3,Deep,Loamy,Slightly alkaline,Moderately well,Very gently sloping,"MULTIPOLYGON (((3715922.076 4528721.605, 37158..."
4,Deep,Loamy,Moderately alkaline,Well,Level to nearly level,"MULTIPOLYGON (((3702439.719 4528235.194, 37029..."


In [2]:
#List the columns
columns = soil_prop_gdf.columns
print(columns)

Index(['Depth', 'Surface_te', 'pH', 'Drainage', 'Slope', 'geometry'], dtype='object')


In [3]:
# Extract and print unique values for each column except 'geometry'
for column in columns:
    # exclude column geometry
    if column != 'geometry':
        unique_values = soil_prop_gdf[column].unique()
        print(f"{column} = {', '.join(map(str, unique_values))}")



Depth = Deep
Surface_te = Loamy, Sandy
pH = Slightly alkaline, Neutral, Moderately alkaline, Strongly alkaline
Drainage = Well, Moderately well, Somewhat excessive
Slope = Level to nearly level, Very gently sloping


---
***AGRICULTURAL SUITABILITY APPLICATION FRAMEWORK***

#### STEP 1: Understanding the suitability of each soil property
***Parameters and Classes***

|***Parameter*** |***Classes***|***Suitability***|
|:-----|:----:|----:|
|Depth |Deep |Suitable for most crops. |
|Surface_te  |Loamy |Highly suitable.|
| |Sandy |Moderately suitable.|
|pH |Neutral |Highly suitable.|
| |Slightly alkaline |Suitable.|
| |Moderately alkaline |Marginally suitable.|
| |Strongly alkaline |Unsuitable.|
|Drainage |Well |Highly suitable.|
| |Moderately well |Suitable.|
| |Somewhat excessive |Marginally suitable.|
|Slope |Level to nearly level |Highly suitable.|
| |Very gently sloping |Suitable.|   

#### STEP 2: Scoring system
Assign scores (e.g., 0-3) based on the suitability level for each parameter:

|***Suitability*** |***Score***|
|:-----|:----:|
|Highly Suitable |3 |
|Suitable  |2 |
|Marginally Suitable |1 |
|Unsuitable |0 |

### STEP 3: Compute final score and classify based on suitability
    1. Calculate the total score by multiplying scores of each parameter the scores of each parameter.
    2. Define thresholds for overall suitability based on the total score.
    We fit the count to normal distribution and categorize the suitability based on total score as below
        a. > (μ+𝜎)      ====> Highly suitable for agriculture.
        b. (μ) to (μ+𝜎) ====> Suitable for agriculture.
        c. (μ-𝜎) to (μ) ====> Marginally suitable; may need amendments.
        d. < (μ-𝜎)      ====> Unsuitable for agriculture.
      where, μ - mean,  𝜎 - standard deviation    

## Assign scores

In [4]:
import pandas as pd

# Assuming soil_prop_gdf is already defined and populated

# Define the scores based on the provided values
scores = {
    'Depth': {'Deep': 3},
    'Surface_te': {'Loamy': 3, 'Sandy': 2},
    'pH': {
        'Neutral': 3, 
        'Slightly alkaline': 2, 
        'Moderately alkaline': 1, 
        'Strongly alkaline': 0
    },
    'Drainage': {
        'Well': 3, 
        'Moderately well': 2, 
        'Somewhat excessive': 1
    },
    'Slope': {
        'Level to nearly level': 3, 
        'Very gently sloping': 2
    }
}

# Create new score columns in the GeoDataFrame and populate them
soil_prop_gdf['depth_score'] = soil_prop_gdf['Depth'].map(scores['Depth'])
soil_prop_gdf['surface_te_score'] = soil_prop_gdf['Surface_te'].map(scores['Surface_te'])
soil_prop_gdf['pH_score'] = soil_prop_gdf['pH'].map(scores['pH'])
soil_prop_gdf['drainage_score'] = soil_prop_gdf['Drainage'].map(scores['Drainage'])
soil_prop_gdf['slope_score'] = soil_prop_gdf['Slope'].map(scores['Slope'])

# Display the updated GeoDataFrame
soil_prop_gdf.head()


,Depth,Surface_te,pH,Drainage,Slope,geometry,depth_score,surface_te_score,pH_score,drainage_score,slope_score
0,Deep,Loamy,Slightly alkaline,Well,Level to nearly level,"MULTIPOLYGON (((3734219.156 4537926.928, 37343...",3,3,2,3,3
1,Deep,Loamy,Neutral,Well,Level to nearly level,"MULTIPOLYGON (((3733766.495 4533437.457, 37336...",3,3,3,3,3
2,Deep,Loamy,Slightly alkaline,Moderately well,Very gently sloping,"MULTIPOLYGON (((3731220.970 4531162.026, 37312...",3,3,2,2,2
3,Deep,Loamy,Slightly alkaline,Moderately well,Very gently sloping,"MULTIPOLYGON (((3715922.076 4528721.605, 37158...",3,3,2,2,2
4,Deep,Loamy,Moderately alkaline,Well,Level to nearly level,"MULTIPOLYGON (((3702439.719 4528235.194, 37029...",3,3,1,3,3


In [5]:
# compute total score
# Ensure the score fields exist in the GeoDataFrame
required_scores = ['depth_score', 'surface_te_score', 'pH_score', 'drainage_score', 'slope_score']
if all(col in soil_prop_gdf.columns for col in required_scores):
    # Create the 'total_score' column by summing the score fields
    soil_prop_gdf['total_score'] = (
        soil_prop_gdf['depth_score'] *
        soil_prop_gdf['surface_te_score'] *
        soil_prop_gdf['pH_score'] *
        soil_prop_gdf['drainage_score'] *
        soil_prop_gdf['slope_score']
    )
else:
    print("One or more score columns are missing in the GeoDataFrame.")

# Display the updated GeoDataFrame
soil_prop_gdf.head()


,Depth,Surface_te,pH,Drainage,Slope,geometry,depth_score,surface_te_score,pH_score,drainage_score,slope_score,total_score
0,Deep,Loamy,Slightly alkaline,Well,Level to nearly level,"MULTIPOLYGON (((3734219.156 4537926.928, 37343...",3,3,2,3,3,162
1,Deep,Loamy,Neutral,Well,Level to nearly level,"MULTIPOLYGON (((3733766.495 4533437.457, 37336...",3,3,3,3,3,243
2,Deep,Loamy,Slightly alkaline,Moderately well,Very gently sloping,"MULTIPOLYGON (((3731220.970 4531162.026, 37312...",3,3,2,2,2,72
3,Deep,Loamy,Slightly alkaline,Moderately well,Very gently sloping,"MULTIPOLYGON (((3715922.076 4528721.605, 37158...",3,3,2,2,2,72
4,Deep,Loamy,Moderately alkaline,Well,Level to nearly level,"MULTIPOLYGON (((3702439.719 4528235.194, 37029...",3,3,1,3,3,81


In [6]:
#compute mean and std dev of total score

In [7]:
# Calculate the mean of the total_score column
mean_total_score = soil_prop_gdf['total_score'].mean()

# Calculate the standard deviation of the total_score column
std_total_score = soil_prop_gdf['total_score'].std()

# Display the results
print(f"Mean of Total Score: {mean_total_score}")
print(f"Standard Deviation of Total Score: {std_total_score}")


Mean of Total Score: 80.71875
Standard Deviation of Total Score: 63.22016541038254


In [8]:
#sum is (μ+𝜎)
sum = mean_total_score + std_total_score
#diff is (μ-𝜎)
diff= mean_total_score - std_total_score
print(sum, diff)

143.93891541038255 17.498584589617458


In [9]:
# classify suitability
# Define a function to classify suitability based on the total_score
def classify_suitability(score):
    if score >= sum:
        return "Highly suitable for agriculture"
    elif mean_total_score <= score <= sum:
        return "Moderately Suitable for agriculture."
    elif diff <= score <= mean_total_score:
        return "Marginally suitable; may need amendments."
    else:
        return "Unsuitable for agriculture"

# Create the 'suitability' column using the classify_suitability function
soil_prop_gdf['suitability'] = soil_prop_gdf['total_score'].apply(classify_suitability)

# Display the updated GeoDataFrame
soil_prop_gdf.head()


,Depth,Surface_te,pH,Drainage,Slope,geometry,depth_score,surface_te_score,pH_score,drainage_score,slope_score,total_score,suitability
0,Deep,Loamy,Slightly alkaline,Well,Level to nearly level,"MULTIPOLYGON (((3734219.156 4537926.928, 37343...",3,3,2,3,3,162,Highly suitable for agriculture
1,Deep,Loamy,Neutral,Well,Level to nearly level,"MULTIPOLYGON (((3733766.495 4533437.457, 37336...",3,3,3,3,3,243,Highly suitable for agriculture
2,Deep,Loamy,Slightly alkaline,Moderately well,Very gently sloping,"MULTIPOLYGON (((3731220.970 4531162.026, 37312...",3,3,2,2,2,72,Marginally suitable; may need amendments.
3,Deep,Loamy,Slightly alkaline,Moderately well,Very gently sloping,"MULTIPOLYGON (((3715922.076 4528721.605, 37158...",3,3,2,2,2,72,Marginally suitable; may need amendments.
4,Deep,Loamy,Moderately alkaline,Well,Level to nearly level,"MULTIPOLYGON (((3702439.719 4528235.194, 37029...",3,3,1,3,3,81,Moderately Suitable for agriculture.


In [10]:
# to display in qgis and check save as geojson
geojson_file = os.path.join(basepath, "agri_suit.geojson")
soil_prop_gdf.to_file(geojson_file, driver="GeoJSON")

## Visualize on leafmap

In [ ]:
import leafmap.foliumap as leafmap
import folium
import geopandas as gpd
import os
from IPython.display import display, HTML  # To display the map inline

# Convert the GeoDataFrame to EPSG:4326
soil_prop_gdf = soil_prop_gdf.to_crs(epsg=4326)

# Check the unique values in the 'suitability' column to ensure correct matching
print("Unique values in suitability column:", soil_prop_gdf['suitability'].unique())

# Define the path for the temporary GeoJSON file
geojson_file = "temp_soil_prop_gdf.geojson"

# Save the GeoDataFrame as a GeoJSON file
soil_prop_gdf.to_file(geojson_file, driver="GeoJSON")

# Define a color mapping for each suitability class, ensuring exact matching with GeoDataFrame values
color_map = {
    "Highly suitable for agriculture": "green",
    "Moderately Suitable for agriculture.": "blue",
    "Marginally suitable; may need amendments.": "orange",
    "Unsuitable for agriculture": "red"
}

# Function to determine color based on the suitability class
def get_color(feature):
    suitability = feature['properties']['suitability']
    # Match the color based on the exact suitability string
    color = color_map.get(suitability, "gray")  # Default to gray if not found
    return {
        'fillColor': color,
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.6
    }

# Calculate the bounds of the GeoDataFrame to set the initial view
bounds = soil_prop_gdf.total_bounds  # [minx, miny, maxx, maxy]
map_center = [(bounds[1] + bounds[3]) / 2, (bounds[0] + bounds[2]) / 2]

# Create a folium map centered on the data and fit bounds
m = leafmap.Map(center=map_center, zoom=5, draw_control=False)
m.fit_bounds([[bounds[1], bounds[0]], [bounds[3], bounds[2]]])

# Add the GeoJSON to the map with proper styling and popups, and name the layer
geo_json_layer = folium.GeoJson(
    geojson_file,
    name="Agricultural Suitability",
    style_function=get_color,
    tooltip=folium.GeoJsonTooltip(
        fields=[col for col in soil_prop_gdf.columns if col != 'geometry'],
        aliases=[col.capitalize() for col in soil_prop_gdf.columns if col != 'geometry']
    )
)

# Disable the default rectangle drawing interaction when clicking on features
geo_json_layer.add_child(
    folium.features.GeoJsonPopup(
        fields=[col for col in soil_prop_gdf.columns if col != 'geometry'],
        aliases=[col.capitalize() for col in soil_prop_gdf.columns if col != 'geometry']
    )
)

# Add GeoJSON layer to the map without the unwanted drawing effect
geo_json_layer.add_to(m)

# Add a styled title to the top middle of the map
title_html = '''
<div style="position: fixed;
     top: 10px; left: 50%; transform: translate(-50%, 0);
     z-index: 9999; font-size: 20px;
     background-color: white; padding: 10px; border: 2px solid black; border-radius: 5px;">
     <b>Agricultural Suitability from Soil Properties in Sonipat District</b>
</div>
'''
m.get_root().html.add_child(folium.Element(title_html))

# Add a custom legend for the symbology positioned at the bottom right corner
legend_html = """
<div style="position: fixed;
     bottom: 50px; right: 50px; width: 300px; height: 150px;
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color: white; padding: 20px;">
     <b>Agricultural Suitability</b><br>
     <i style="background: green; width: 10px; height: 10px; display: inline-block;"></i> Highly suitable for agriculture.<br>
     <i style="background: blue; width: 10px; height: 10px; display: inline-block;"></i> Moderately Suitable for agriculture.<br>
     <i style="background: orange; width: 10px; height: 10px; display: inline-block;"></i> Marginally suitable; may need amendments.<br>
     <i style="background: red; width: 10px; height: 10px; display: inline-block;"></i> Unsuitable for agriculture.<br>
</div>
"""
m.get_root().html.add_child(folium.Element(legend_html))

# Display the map inline in Colab
m


Unique values in suitability column: ['Highly suitable for agriculture'
 'Marginally suitable; may need amendments.'
 'Moderately Suitable for agriculture.' 'Unsuitable for agriculture']
